In [1]:
# import gspread
# import cantera as ct
# from cantera import ck2cti
# from oauth2client.service_account import ServiceAccountCredentials
import matplotlib as mpl
mpl.use("pgf")
# from palettable.tableau import Tableau_10
# from cycler import cycler
import numpy as np
from cycler import cycler
pgf_with_pdflatex = {
    "pgf.texsystem": "pdflatex",
    "pgf.preamble": [
         r"\usepackage[utf8x]{inputenc}",
         r"\usepackage[T1]{fontenc}",
         r"\usepackage{mathptmx}",
         r"\usepackage{mathtools}",
#          r"\usepackage{biblatex}",
         ],
    "text.usetex": True,
    "figure.figsize": [4, 3.5],
    "axes.labelsize": 12,
    "axes.labelpad": 5.0,
    "font.size": 12,
    "font.family": "serif",
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "lines.linewidth": 1.5,
#     "axes.prop_cycle": cycler(color=Tableau_10.mpl_colors),
    "xtick.major.size": 5,
    "xtick.major.width": 1,
    "xtick.minor.size": 2.5,
    "xtick.minor.width": 1,
    "ytick.major.size": 5,
    "ytick.major.width": 1,
    "ytick.minor.size": 2.5,
    "ytick.minor.width": 1,
    "axes.prop_cycle": cycler('color', ['#1f77b4', '#ff7f0e', '#2ca02c'])
}
mpl.rcParams.update(pgf_with_pdflatex)
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, AutoMinorLocator
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
# from palettable.tableau import Tableau_10
import pickle
# from scipy.optimize import curve_fit
from scipy.interpolate import UnivariateSpline
base = Path('simulation-comparison')

In [2]:
markers = ['o', 's', 'd', '^', 'x', '+', 'v', '<', '>', '*']
def mfunc(syms):
    while True:
        for s in syms:
            yield s

In [3]:
# scope = ['https://spreadsheets.google.com/feeds']
# credentials = ServiceAccountCredentials.from_json_keyfile_name('methyl-valerate-78135e2c2c7c.json', scope)
# gc = gspread.authorize(credentials)
# ws = gc.open_by_key('1UFdGzEyx7PsKprPybi-lkbuWYS8BAS-cBm2IKSKY-UA').sheet1
# recs = ws.get_all_records(empty2zero=True)
# data = pd.DataFrame(recs)
# data.to_pickle(str(base/'data-file.pkl'))

In [4]:
data = pd.read_pickle(str(base/'data-file.pkl'))

phi_200_15_bar = data[(data['Equivalence Ratio'] == 2.00) &
                      (np.isclose(data['Compressed Pressure (bar)'], 15, rtol=1E-2))][::-1]
phi_100_15_bar = data[(data['Equivalence Ratio'] == 1.00) &
                      (np.isclose(data['Compressed Pressure (bar)'], 15, rtol=1E-2))][::-1]
phi_100_30_bar = data[(data['Equivalence Ratio'] == 1.00) &
                      (np.isclose(data['Compressed Pressure (bar)'], 30, rtol=1E-2))][::-1]
phi_050_15_bar = data[(data['Equivalence Ratio'] == 0.50) &
                      (np.isclose(data['Compressed Pressure (bar)'], 15, rtol=1E-1))][::-1]
phi_050_30_bar = data[(data['Equivalence Ratio'] == 0.50) &
                      (np.isclose(data['Compressed Pressure (bar)'], 30, rtol=1E-2))][::-1]
phi_025_15_bar = data[(data['Equivalence Ratio'] == 0.25) &
                      (np.isclose(data['Compressed Pressure (bar)'], 15, rtol=1E-2))][::-1]
phi_025_30_bar = data[(data['Equivalence Ratio'] == 0.25) &
                      (np.isclose(data['Compressed Pressure (bar)'], 30, rtol=1E-2))][::-1]

In [19]:
fig, ax_phi_100 = plt.subplots()

# Set the axis labels
# fig.text(0.04, 0.5, 'Ignition Delay, ms', verticalalignment='center', rotation='vertical')
# fig.text(0.5, 0.04, '$1000/T_C$, 1/K', horizontalalignment='center')

ax_phi_100.set_ylabel('Ignition Delay, ms')
ax_phi_100.set_xlabel('$1000/T_C$, 1/K')

plot_opts = {'markersize': 7, 'elinewidth': 1.5, 'capthick': 1.5, 'fmt': 'o'}
plot_opts_15 = plot_opts
plot_opts_30 = {**plot_opts, 'markerfacecolor': 'none', 'markeredgewidth': 1.0}
plot_opts_sim_mark = {'markersize': 5, 'marker': 'o', 'linestyle': ''}
plot_opts_sim_mark_15 = {**plot_opts_sim_mark}
plot_opts_sim_mark_30 = {**plot_opts_sim_mark, 'markerfacecolor': 'none', 'markeredgewidth': 1.0}
plot_opts_sim_line_15 = {'linestyle': '-'}
plot_opts_sim_det = {}
plot_opts_sim_line_30 = {'linestyle': '--'}
plot_opts_sim_rmg = {}


# Set the scale of the y axes
# ax_phi_025.set_yscale('log')
# ax_phi_050.set_yscale('log')
ax_phi_100.set_yscale('log')
# ax_phi_200.set_yscale('log')

# Set the formatting of the y tick labels
# ax_phi_025.yaxis.set_major_formatter(FormatStrFormatter('%d'))
# ax_phi_050.yaxis.set_major_formatter(FormatStrFormatter('%d'))

ax_phi_100.yaxis.set_major_formatter(FormatStrFormatter('%d'))
# ax_phi_200.yaxis.set_major_formatter(FormatStrFormatter('%d'))

# Create the temperature axes on the top
def convert_inv_temp(temps):
    """Convert a list of temperatures to inverse temperature"""
    return [1000.0/temp for temp in temps]

# Set the major tick marks in the temperature scale and convert to inverse scale
major_temps = np.arange(1100, 600, -100)
major_ticks = convert_inv_temp(major_temps)

# Set the interval for the minor ticks and compute the minor ticks
minor_interval = 20
minor_ticks = []
for maj in major_temps:
    minor_ticks.extend(convert_inv_temp([maj - i*minor_interval for i in range(5)]))

# $\phi$=2.00

# $\phi$=1.00

In [20]:
phi_100_15_rmg = pd.read_csv(str(base/'phi=1.0/15-bar/chem_rmg_results.csv')).dropna(subset=['Simulated Overall Delay (ms)']).sort_values(by='1000/Tc (1/K)')
phi_100_15_det = pd.read_csv(str(base/'phi=1.0/15-bar/Detailed_Model_results.csv')).dropna(subset=['Simulated Overall Delay (ms)']).sort_values(by='1000/Tc (1/K)')
phi_100_30_rmg = pd.read_csv(str(base/'phi=1.0/30-bar/chem_rmg_results.csv')).dropna(subset=['Simulated Overall Delay (ms)']).sort_values(by='1000/Tc (1/K)')
phi_100_30_det = pd.read_csv(str(base/'phi=1.0/30-bar/Detailed_Model_results.csv')).dropna(subset=['Simulated Overall Delay (ms)']).sort_values(by='1000/Tc (1/K)')
# phi_100_rmg = pickle.load(open(str(base/'phi_100_rmg.pkl'), 'rb'))
# phi_100_red = pickle.load(open(str(base/'phi_100_red.pkl'), 'rb'))
# phi_100_det = pickle.load(open(str(base/'phi_100_det.pkl'), 'rb'))
# ax_phi_100.plot(1000.0/phi_100_rmg[15.0E5][:, 0], phi_100_rmg[15.0E5][:, 1]*1000.0, label='RMG')
# ax_phi_100.plot(1000.0/phi_100_red[15.0E5][:, 0], phi_100_red[15.0E5][:, 1]*1000.0, label='RED')
# ax_phi_100.plot(1000.0/phi_100_det[15.0E5][:, 0], phi_100_det[15.0E5][:, 1]*1000.0, label='DET')
phi_100_15_rmg_temps = np.linspace(phi_100_15_rmg['1000/Tc (1/K)'].iloc[0], phi_100_15_rmg['1000/Tc (1/K)'].iloc[-1])
phi_100_15_det_temps = np.linspace(phi_100_15_det['1000/Tc (1/K)'].iloc[0], phi_100_15_det['1000/Tc (1/K)'].iloc[-1])
ax_phi_100.errorbar(phi_100_15_bar['1000/Tc (1/K)'], phi_100_15_bar['Ignition Delay (ms)'], yerr=np.array(phi_100_15_bar['Ignition Delay Error (ms)']), **plot_opts_15)
# ax_phi_100.plot(phi_100_15_rmg['1000/Tc (1/K)'], phi_100_15_rmg['Simulated Overall Delay (ms)'], **plot_opts_sim_mark_15, **plot_opts_sim_rmg)
spl = UnivariateSpline(phi_100_15_rmg['1000/Tc (1/K)'], np.log(phi_100_15_rmg['Simulated Overall Delay (ms)']))
ax_phi_100.plot(phi_100_15_rmg_temps, np.exp(spl(phi_100_15_rmg_temps)), **plot_opts_sim_line_15)
# ax_phi_100.plot(phi_100_15_det['1000/Tc (1/K)'], phi_100_15_det['Simulated Overall Delay (ms)'], **plot_opts_sim_mark_15, **plot_opts_sim_det)
spl = UnivariateSpline(phi_100_15_det['1000/Tc (1/K)'], np.log(phi_100_15_det['Simulated Overall Delay (ms)']))
ax_phi_100.plot(phi_100_15_det_temps, np.exp(spl(phi_100_15_det_temps)), **plot_opts_sim_line_15)

# ax_phi_100.plot(1000.0/phi_100_rmg[30.0E5][:, 0], phi_100_rmg[30.0E5][:, 1]*1000.0, label='RMG')
# ax_phi_100.plot(1000.0/phi_100_red[30.0E5][:, 0], phi_100_red[30.0E5][:, 1]*1000.0, label='RED')
# ax_phi_100.plot(1000.0/phi_100_det[30.0E5][:, 0], phi_100_det[30.0E5][:, 1]*1000.0, label='DET')

# ax_phi_100.errorbar(phi_100_30_bar['1000/Tc (1/K)'], phi_100_30_bar['First Stage Delay (ms)'], yerr=np.array(phi_100_30_bar['First Stage Error (ms)']), fmt=next(mark), **plot_opts)
phi_100_30_rmg_temps = np.linspace(phi_100_30_rmg['1000/Tc (1/K)'].iloc[0], phi_100_30_rmg['1000/Tc (1/K)'].iloc[-1])
phi_100_30_det_temps = np.linspace(phi_100_30_det['1000/Tc (1/K)'].iloc[0], phi_100_30_det['1000/Tc (1/K)'].iloc[-1])
ax_phi_100.errorbar(phi_100_30_bar['1000/Tc (1/K)'], phi_100_30_bar['Ignition Delay (ms)'], yerr=np.array(phi_100_30_bar['Ignition Delay Error (ms)']), **plot_opts_30)
# ax_phi_100.plot(phi_100_30_rmg['1000/Tc (1/K)'], phi_100_30_rmg['Simulated Overall Delay (ms)'], **plot_opts_sim_mark_30, **plot_opts_sim_rmg)
spl = UnivariateSpline(phi_100_30_rmg['1000/Tc (1/K)'], np.log(phi_100_30_rmg['Simulated Overall Delay (ms)']))
ax_phi_100.plot(phi_100_30_rmg_temps, np.exp(spl(phi_100_30_rmg_temps)), **plot_opts_sim_line_30)
# ax_phi_100.plot(phi_100_30_det['1000/Tc (1/K)'], phi_100_30_det['Simulated Overall Delay (ms)'], **plot_opts_sim_mark_30, **plot_opts_sim_det)
spl = UnivariateSpline(phi_100_30_det['1000/Tc (1/K)'], np.log(phi_100_30_det['Simulated Overall Delay (ms)']))
ax_phi_100.plot(phi_100_30_det_temps, np.exp(spl(phi_100_30_det_temps)), **plot_opts_sim_line_30)

ax_phi_100.set_ylim(1, 150)
# ax_phi_100.set_xlim(1.03, 1.5)
ax_phi_100.xaxis.set_minor_locator(AutoMinorLocator(4))

ax_phi_100.text(0.04, 0.90, r'$P_C = 15\ \text{bar}$', transform=ax_phi_100.transAxes)
ax_phi_100.text(0.55, 0.60, r'$P_C = 30\ \text{bar}$', transform=ax_phi_100.transAxes)

# pc_15_bar = Line2D([], [], color='black', marker='o', linestyle='-', label=r'$P_C = 15\ \text{bar}')
# pc_30_bar = Line2D([], [], color='black', marker='o', linestyle='--', markerfacecolor='none', label=r'$P_C = 30\ \text{bar}')
expt = Patch(color='#1f77b4', label='Experimental Data')
rmg = Patch(color='#ff7f0e', label='RMG Model')
det = Patch(color='#2ca02c', label=r"Di\'evart et al.\ \cite{Dievart2013} Model")

ax_phi_100.legend(handles=[expt, det, rmg], loc='upper right',fancybox=False, framealpha=0.0)

ax_100_temp = ax_phi_100.twiny()

# Set the ticks on the axis. Note that the limit setting must be present and must be after setting the ticks
# so that the scale is correct
ax_100_temp.set_xticks(major_ticks)
ax_100_temp.set_xticks(minor_ticks, minor=True)
ax_100_temp.set_xticklabels(['{:d} K'.format(temp) for temp in major_temps])
ax_100_temp.set_xlim(ax_phi_100.get_xlim());

In [22]:
fig.savefig('simulation-comparison.pgf', bbox_inches='tight')

# $\phi$=0.50

# $\phi$=0.25

Data plotting:
15 bar overall
15 bar RMG
15 bar DET

30 bar overall
30 bar first stage
30 bar RMG
30 bar DET